In [1]:
# import sys
# !{sys.executable} -m pip install --upgrade pip
# !{sys.executable} -m pip install -r requirements.txt

In [115]:

import numpy as np
import pandas as pd
import datetime
import random
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 18})
plt.style.use('ggplot')
import seaborn as sns
from scipy import stats
from sklearn.impute import SimpleImputer
import shap
from sklearn.preprocessing import StandardScaler
warnings.filterwarnings("ignore", category=FutureWarning)


In [116]:
df = pd.read_csv(r"./GIT_data/data_set.csv")

print("Number of df samples are",df.shape)

categorical_features = ['State_Factor', 'building_class', 'facility_type']
numerical_features=df.select_dtypes('number').columns


Number of df samples are (75757, 64)


In [117]:
df.head()

,Year_Factor,State_Factor,building_class,facility_type,floor_area,year_built,energy_star_rating,ELEVATION,january_min_temp,january_avg_temp,...,days_above_80F,days_above_90F,days_above_100F,days_above_110F,direction_max_wind_speed,direction_peak_wind_speed,max_wind_speed,days_with_fog,site_eui,id
0,1,State_1,Commercial,Grocery_store_or_food_market,61242.0,1942.0,11.0,2.4,36,50.5,...,14,0,0,0,1.0,1.0,1.0,NaN,248.682615,0
1,1,State_1,Commercial,Warehouse_Distribution_or_Shipping_center,274000.0,1955.0,45.0,1.8,36,50.5,...,14,0,0,0,1.0,NaN,1.0,12.0,26.500150,1
2,1,State_1,Commercial,Retail_Enclosed_mall,280025.0,1951.0,97.0,1.8,36,50.5,...,14,0,0,0,1.0,NaN,1.0,12.0,24.693619,2
3,1,State_1,Commercial,Education_Other_classroom,55325.0,1980.0,46.0,1.8,36,50.5,...,14,0,0,0,1.0,NaN,1.0,12.0,48.406926,3
4,1,State_1,Commercial,Warehouse_Nonrefrigerated,66000.0,1985.0,100.0,2.4,36,50.5,...,14,0,0,0,1.0,1.0,1.0,NaN,3.899395,4


In [5]:
# plt.figure(figsize = (25,11))
# sns.heatmap(df.isna().values, xticklabels=df.columns)
# plt.title("Missing values in  Data", size=20)

In [119]:
def angle_to_direction(angle):
    # Ensure the angle is within the range [0, 360)
    normalized_angle = angle % 360

    # Define directional ranges
    directions = {
        (337.5, 22.5): 'north',
        (22.5, 67.5): 'northeast',
        (67.5, 112.5): 'east',
        (112.5, 157.5): 'southeast',
        (157.5, 202.5): 'south',
        (202.5, 247.5): 'southwest',
        (247.5, 292.5): 'west',
        (292.5, 337.5): 'northwest'
    }

    # Determine the direction based on the angle
    for angle_range, direction_name in directions.items():
        if normalized_angle >= angle_range[0] and normalized_angle < angle_range[1]:
            return direction_name

    # If the angle is not within any defined range, return None or handle it accordingly
    return 'unknown'

# Example usage:
angle = -1  # Replace this with your angle
direction = angle_to_direction(angle)
print(f"The direction for an angle of {angle} degrees is: {direction}")


The direction for an angle of -1 degrees is: unknown


In [136]:
# Change wind speed angle to directions
df['direction_max_wind_speed_cate'] = df['direction_max_wind_speed'].apply(angle_to_direction)
df['direction_peak_wind_speed_cate'] = df['direction_peak_wind_speed'].apply(angle_to_direction)

In [122]:
missing_columns = [col for col in df.columns if df[col].isnull().any()]
missingvalues_count =df.isna().sum()
missingValues_df = pd.DataFrame(missingvalues_count.rename('Null Values Count')).loc[missingvalues_count.ne(0)]
missingValues_df.style.background_gradient(cmap="Pastel1")

df['year_built'] =df['year_built'].replace(np.nan, 2022)
null_col=['energy_star_rating','direction_max_wind_speed','direction_peak_wind_speed','max_wind_speed','days_with_fog']
imputer = SimpleImputer()
imputer.fit(df[null_col])
data_transformed = imputer.transform(df[null_col])
df[null_col] = pd.DataFrame(data_transformed)

In [123]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75757 entries, 0 to 75756
Data columns (total 66 columns):
 #   Column                          Non-Null Count  Dtype  
---  ------                          --------------  -----  
 0   Year_Factor                     75757 non-null  int64  
 1   State_Factor                    75757 non-null  object 
 2   building_class                  75757 non-null  object 
 3   facility_type                   75757 non-null  object 
 4   floor_area                      75757 non-null  float64
 5   year_built                      75757 non-null  float64
 6   energy_star_rating              75757 non-null  float64
 7   ELEVATION                       75757 non-null  float64
 8   january_min_temp                75757 non-null  int64  
 9   january_avg_temp                75757 non-null  float64
 10  january_max_temp                75757 non-null  int64  
 11  february_min_temp               75757 non-null  int64  
 12  february_avg_temp               

In [9]:
# plt.figure(figsize = (25,11))
# sns.heatmap(df.isna().values, xticklabels=df.columns)
# plt.title("Missing values in training Data", size=20)

In [124]:
categorical_features

['State_Factor', 'building_class', 'facility_type']

In [125]:
categorical_features_new = categorical_features + ['direction_max_wind_speed_cate', 'direction_peak_wind_speed_cate']
categorical_features_new

['State_Factor',
 'building_class',
 'facility_type',
 'direction_max_wind_speed_cate',
 'direction_peak_wind_speed_cate']

In [126]:
encoder = OneHotEncoder(sparse=False, categories='auto')
one_hot_facility_type = encoder.fit_transform(df[categorical_features_new])
for i, category in enumerate(encoder.categories_[0]):
  df[f'is_{category}'] = one_hot_facility_type[:, i]

In [128]:
for i, category in enumerate(encoder.categories_[1]):
  df[f'is_{category}'] = one_hot_facility_type[:, i]
for i, category in enumerate(encoder.categories_[2]):
  df[f'is_{category}'] = one_hot_facility_type[:, i]
for i, category in enumerate(encoder.categories_[3]):
  df[f'is_{category}'] = one_hot_facility_type[:, i]
for i, category in enumerate(encoder.categories_[4]):
  df[f'is_{category}'] = one_hot_facility_type[:, i]

In [73]:
# le = OneHotEncoder(sparse=False, categories='auto')#LabelEncoder()
# for col in categorical_features_new:
#     df[col] = le.fit_transform(df[['State_Factor']])

In [129]:
df.head()

,Year_Factor,State_Factor,building_class,facility_type,floor_area,year_built,energy_star_rating,ELEVATION,january_min_temp,january_avg_temp,...,is_Service_Uncategorized,is_Service_Vehicle_service_repair_shop,is_Warehouse_Distribution_or_Shipping_center,is_Warehouse_Nonrefrigerated,is_Warehouse_Refrigerated,is_Warehouse_Selfstorage,is_Warehouse_Uncategorized,is_northwest,is_unknown,is_west
0,1,State_1,Commercial,Grocery_store_or_food_market,61242.0,1942.0,11.0,2.4,36,50.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,1,State_1,Commercial,Warehouse_Distribution_or_Shipping_center,274000.0,1955.0,45.0,1.8,36,50.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,1,State_1,Commercial,Retail_Enclosed_mall,280025.0,1951.0,97.0,1.8,36,50.5,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0
3,1,State_1,Commercial,Education_Other_classroom,55325.0,1980.0,46.0,1.8,36,50.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
4,1,State_1,Commercial,Warehouse_Nonrefrigerated,66000.0,1985.0,100.0,2.4,36,50.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0


In [12]:
# print("Unique Value Count:")
# cols = df.columns.tolist()
# for col in cols:
#   print(col + " = " + str(len(df[col].unique())))

In [13]:
# hist = df.hist(figsize=(20,20),layout=(8,8))

In [14]:
# hist = df[numerical_features].hist(figsize=(20,20),layout=(8,8))

In [15]:
# hist = df[null_col].hist(figsize=(12,8),layout=(2,3))

In [16]:
#4 also same in the missing value part - direction_max_wind_speed + direction_peak_wind_speed

In [17]:
#2 ELEVATION - transformation？
# df[df.ELEVATION < 0]

In [18]:
#1 year_factor - Drop？
# print(f'The number of year_factor:{df.Year_Factor.nunique()}')
# df.Year_Factor.unique()
# The number of year_factor:6
# array([1, 2, 3, 4, 5, 6])

In [112]:
df.describe().style.background_gradient()

,Year_Factor,floor_area,year_built,energy_star_rating,ELEVATION,january_min_temp,january_avg_temp,january_max_temp,february_min_temp,february_avg_temp,february_max_temp,march_min_temp,march_avg_temp,march_max_temp,april_min_temp,april_avg_temp,april_max_temp,may_min_temp,may_avg_temp,may_max_temp,june_min_temp,june_avg_temp,june_max_temp,july_min_temp,july_avg_temp,july_max_temp,august_min_temp,august_avg_temp,august_max_temp,september_min_temp,september_avg_temp,september_max_temp,october_min_temp,october_avg_temp,october_max_temp,november_min_temp,november_avg_temp,november_max_temp,december_min_temp,december_avg_temp,december_max_temp,cooling_degree_days,heating_degree_days,precipitation_inches,snowfall_inches,snowdepth_inches,avg_temp,days_below_30F,days_below_20F,days_below_10F,days_below_0F,days_above_80F,days_above_90F,days_above_100F,days_above_110F,direction_max_wind_speed,direction_peak_wind_speed,max_wind_speed,days_with_fog,site_eui,id,is_State_1,is_State_10,is_State_11,is_State_2,is_State_4,is_State_6,is_State_8
count,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000,75757.000000
mean,4.367755,165983.865858,1953.996726,61.048605,39.506323,11.432343,34.310468,59.054952,11.720567,35.526837,58.486278,21.606281,44.469292,70.897831,32.037171,53.784863,82.661932,44.706179,63.742547,88.987856,51.131763,71.066384,91.400280,60.571604,76.596879,95.543950,58.515068,75.348749,92.572581,50.791689,69.597545,92.171139,38.210911,58.727038,81.135671,28.630397,48.123133,71.202397,22.736777,41.625569,64.496852,1202.250446,4324.957390,42.430651,29.136379,164.862455,56.176705,48.756511,17.447932,4.886532,0.876764,82.709809,14.058701,0.279539,0.002442,66.552675,62.779974,4.190601,109.142051,82.584693,37878.000000,0.074158,0.000198,0.084639,0.064298,0.056760,0.671093,0.048854
std,1.471441,246875.790940,38.139247,23.063738,60.656596,9.381027,6.996108,5.355458,12.577272,8.866697,8.414611,10.004303,6.657142,7.680982,5.577279,2.500473,4.101028,4.200319,2.792919,2.864669,3.415926,2.861681,3.604068,4.177109,4.090275,4.157118,4.441992,3.818895,4.139685,4.290884,3.603773,5.011708,5.093899,2.641300,5.579513,7.153244,4.094207,4.373374,9.014047,5.869386,6.054104,391.561780,824.366727,10.592205,17.810894,175.275869,2.241940,27.587124,14.469435,7.071221,2.894244,25.282913,10.943996,2.252323,0.142140,88.726751,87.226977,4.369629,31.883663,58.255403,21869.306509,0.262030,0.014070,0.278346,0.245284,0.231386,0.469819,0.215563
min,1.000000,943.000000,0.000000,0.000000,-6.400000,-19.000000,10.806452,42.000000,-13.000000,13.250000,38.000000,-9.000000,25.854839,53.000000,15.000000,40.280000,62.000000,23.000000,46.950000,64.000000,30.000000,54.100000,67.000000,37.000000,54.822581,65.000000,31.000000,56.693548,66.000000,26.000000,53.600000,64.000000,18.000000,44.693548,59.000000,4.000000,30.766667,53.000000,-16.000000,23.790323,42.000000,0.000000,398.000000,0.000000,0.000000,0.000000,44.512329,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,1.000000,1.000000,12.000000,1.001169,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,3.000000,62379.000000,1927.000000,56.000000,11.900000,6.000000,29.82758

In [113]:
# drop_all = missing_columns + ["site_eui","id", "Year_Factor"]
# drop_all

In [131]:
import copy
y = df["site_eui"]
train = df.drop(["site_eui","id", "Year_Factor", "direction_max_wind_speed", "direction_peak_wind_speed", "State_Factor", "building_class", "facility_type", 
                 "direction_peak_wind_speed_cate", "direction_max_wind_speed_cate"],axis =1)
trainnames = copy.deepcopy(train)
scaler = StandardScaler()
train = scaler.fit_transform(train)


In [45]:
train

array([[-2.73502847e+00, -1.16308853e+00, -1.11048095e+00, ...,
         4.45712541e-16,  2.77173989e-01,  1.56862898e-01],
       [-2.73502847e+00, -1.16308853e+00,  2.49611589e+00, ...,
        -3.04678581e+00,  2.77173989e-01,  1.56862898e-01],
       [-2.73502847e+00, -1.16308853e+00,  1.89501642e+00, ...,
        -3.04678581e+00,  2.77173989e-01,  1.56862898e-01],
       ...,
       [-1.44431120e+00, -1.16308853e+00, -2.05506583e+00, ...,
         4.45712541e-16,  2.77173989e-01,  1.56862898e-01],
       [-1.44431120e+00, -1.16308853e+00, -2.05506583e+00, ...,
         4.45712541e-16,  2.77173989e-01,  1.56862898e-01],
       [-1.44431120e+00,  8.59779779e-01, -2.22680854e+00, ...,
         4.45712541e-16,  2.77173989e-01,  1.56862898e-01]])

In [132]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train, y, test_size = 0.2, random_state = 50)


In [133]:
X_test_ft=pd.DataFrame(X_test,columns=trainnames.columns)
X_test_ft.shape

(15152, 128)

In [134]:
import xgboost
xgboost_model = xgboost.XGBRegressor(n_estimators=200, learning_rate=0.02, gamma=0, subsample=0.75,
                           colsample_bytree=0.4, max_depth=5)
xgboost_model.fit(X_train,y_train)

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=0.4, device=None, early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=0, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=0.02, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=5, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=200, n_jobs=None,
             num_parallel_tree=None, random_state=None, ...)

In [135]:
from sklearn.model_selection import GridSearchCV

# Define the hyperparameter grid
param_grid = {
    'n_estimators': [100, 250],
    'learning_rate': [0.01, 0.05],
    'max_depth': [3, 4],
    'subsample': [0.6, 0.9],
    'colsample_bytree': [0.2, 0.5],
    'gamma': [0.01, 0.05]
}

# Create an XGBoost regressor
xgboost_model = xgboost.XGBRegressor()

# Create GridSearchCV
grid_search = GridSearchCV(estimator=xgboost_model, param_grid=param_grid,
                           scoring='neg_mean_squared_error', cv=5, verbose=1, n_jobs=-1)

# Fit the model to the training data
grid_search.fit(X_train, y_train)

# Print the best hyperparameters
print("Best Hyperparameters:", grid_search.best_params_)

# Get the best model
best_model = grid_search.best_estimator_


# Make predictions on the test set using the best model
y_pred = best_model.predict(X_test)

# Calculate RMSE
rmse = np.sqrt(mean_squared_error(y_test, y_pred))
print("Root Mean Squared Error (RMSE) on Test Set:", rmse)

Fitting 5 folds for each of 64 candidates, totalling 320 fits


/Users/xuchen/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/xuchen/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype) or not is_extension_array_dtype(pd_dtype):
/Users/xuchen/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:605: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_sparse(pd_dtype):
/Users/xuchen/anaconda3/lib/python3.10/site-packages/sklearn/utils/validation.py:614: FutureWarning: is_sparse is deprecated and will be removed in a future version. Check `isinstance(dtype, pd.SparseDtype)` instead.
  if is_spar

Best Hyperparameters: {'colsample_bytree': 0.5, 'gamma': 0.01, 'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 250, 'subsample': 0.9}
Root Mean Squared Error (RMSE) on Test Set: 42.94350134575407


In [27]:
# Best Hyperparameters: {'colsample_bytree': 0.5, 'gamma': 0.01, 'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 250, 'subsample': 0.9}
# Root Mean Squared Error (RMSE) on Test Set: 43.00527616459719

# Best Hyperparameters: {'colsample_bytree': 0.5, 'gamma': 0.01, 'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 250, 'subsample': 0.9}
# Root Mean Squared Error (RMSE) on Test Set: 43.00527616459719

# Drop all the missing values
# Best Hyperparameters: {'colsample_bytree': 0.5, 'gamma': 0.01, 'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 250, 'subsample': 0.9}
# Root Mean Squared Error (RMSE) on Test Set: 42.970687172969185

# Best Hyperparameters: {'colsample_bytree': 0.5, 'gamma': 0.01, 'learning_rate': 0.05, 'max_depth': 4, 'n_estimators': 250, 'subsample': 0.9}
# Root Mean Squared Error (RMSE) on Test Set: 42.94350134575407

In [51]:
import eli5
from eli5.sklearn import PermutationImportance

perm = PermutationImportance(best_model, random_state=1).fit(X_test, y_test)
eli5.show_weights(perm, feature_names = trainnames.columns.tolist(), top=63)

Weight,Feature
0.3741 ± 0.0196,energy_star_rating
0.2957 ± 0.0058,facility_type
0.1230 ± 0.0212,floor_area
0.0441 ± 0.0034,building_class
0.0212 ± 0.0025,State_Factor
0.0201 ± 0.0022,year_built
0.0089 ± 0.0019,snowdepth_inches
0.0084 ± 0.0010,february_avg_temp
0.0077 ± 0.0011,january_avg_temp
0.0054 ± 0.0008,ELEVATION
